In [1]:
import pandas as pd
import numpy as np

In [104]:
path = r'Experiments/MultiplyTemperature/Exp0(0)/Data'
df1 = pd.read_csv(f'{path}/1.csv',index_col=0)
df2 = pd.read_csv(f'{path}/2.csv',index_col=0)
df3 = pd.read_csv(f'{path}/3.csv',index_col=0)
df4 = pd.read_csv(f'{path}/4.csv',index_col=0)

In [105]:
rules= {0:'OK',1:'processor_sweep',2:'value_combine',3:'error'}
col_rules={'Viscosity_mark':'Viscosity_verbose','Temperature_mark':'Temperature_verbose'} 

In [106]:
df1 = df1.rename(columns=col_rules)
df1[['Viscosity_verbose','Temperature_verbose']] = df1[['Viscosity_verbose','Temperature_verbose']].replace(rules)

df2 = df2.rename(columns=col_rules)
df2[['Viscosity_verbose','Temperature_verbose']] = df2[['Viscosity_verbose','Temperature_verbose']].replace(rules)
df2['time'] =df2['time']+ df1['time'].max()+1

df3 = df3.rename(columns=col_rules)
df3[['Viscosity_verbose','Temperature_verbose']] = df3[['Viscosity_verbose','Temperature_verbose']].replace({np.nan:'OK'})
df3['time'] =df3['time']+ df2['time'].max()+1

df4 = df4.rename(columns=col_rules)
df4[['Viscosity_verbose','Temperature_verbose']] = df4[['Viscosity_verbose','Temperature_verbose']].replace({np.nan:'OK'})
df4['time'] =df4['time']+ df3['time'].max()+1

In [107]:
df = pd.concat([df1,df2,df3,df4])

# time plots

In [10]:
import plotly 
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [11]:
%matplotlib qt

In [108]:
plot_rules = {'processor_sweep':2, 'OK':0, 'error':4, 'value_combine':3,
       'inner_processor_check':1}
df[['Viscosity_verbose','Temperature_verbose']] = df[['Viscosity_verbose','Temperature_verbose']].replace(plot_rules)
df_cleaned= df.dropna(subset=['Viscosity','Temperature'])
df['time']=df['time']/60

In [110]:
dfs =df_cleaned


fig,ax_v =plt.subplots()
ax_T = ax_v.twinx()

# df_masked = dfs[(dfs['Temperature_verbose']<3) & (dfs ['Viscosity_verbose']<3)]
df_masked= dfs

ax_v.scatter(df_masked['time'],df_masked['Viscosity'],color='red',marker='.')
ax_v.set_ylabel('Viscosity',color='red')
ax_v.set_ylim((0,350))


ax_T.scatter(df_masked['time'],df_masked['Temperature'],color='blue',marker='.')
ax_T.set_ylabel('Temperature',color='blue')
ax_T.set_ylim((13.5,40.5))


# df_masked = dfs[(dfs['Temperature_verbose']>=3) | (dfs ['Viscosity_verbose']>=3)]
# ax_v.scatter(df_masked['time'],df_masked['Viscosity'],color='black',marker='s')
# ax_T.scatter(df_masked['time'],df_masked['Temperature'],color='gray',marker='o')

(13.5, 40.5)